In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Single-channel T-matrix 3D Riemann sheets
TR-999
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=4) of the PDG.
+++
🚧&nbsp;[compwa.github.io#204](https://github.com/ComPWA/compwa.github.io/pull/204)
:::
::::

# Riemann sheets for single-channel T matrix

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.18.0 sympy==1.12

:::{note}
In the following example we assume zero angular momentum ($L=0$).
:::

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

## Square root definition

We define a special expression class for a square root where you can choose the sign for negative arguments. This can be used later for the phase space factor&nbsp;$\rho$.

In [ ]:
@unevaluated(real=False)
class SignedSqrt(sp.Expr):
    z: Any
    _latex_repr_ = R"\sqrt[+]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)


@unevaluated
class PosArg(sp.Expr):
    z: Any
    _latex_repr_ = R"\arg^+\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg + 2 * sp.pi, arg < -2 * sp.pi),
            (arg - 2 * sp.pi, arg > 0),
            (arg, True),
        )


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

## Phase space factor definitions

### Standard phase space

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^+_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

### Chew-Mandelstam function  

In [ ]:
@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m_a: Any
    m_b: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (s * 4))


rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
Math(aslatex({e: e.doit(deep=False) for e in [rho_cm_expr, cm_expr, q_expr]}))

### Alternative definition of Chew-Mandelstam

By J. L. Basdevant and Edmond L. Berger
(Phys. Rev. D 16, 657 – Published 1 August 1977)

In [ ]:
@unevaluated(real=False)
class ChewMandelstamBB(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\text{{CM}}_\text{{BB}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        rho = PhaseSpaceFactor(s, m1, m2)
        return (
            1 / sp.pi
            + (m1**2 - m2**2) / (sp.pi * s) * sp.log(m2 / m1)
            - (m1**2 + m2**2) / (sp.pi * (m1**2 - m2**2)) * sp.log(m2 / m1)
            + rho
            / sp.pi
            * sp.log(
                (sp.sqrt((m1 + m2) ** 2 - s) - sp.sqrt((m1 - m2) ** 2 - s))
                / (sp.sqrt((m1 + m2) ** 2 - s) + sp.sqrt((m1 - m2) ** 2 - s))
            )
        )


rho_bb_expr = ChewMandelstamBB(s, m1, m2)
Math(aslatex({rho_bb_expr: rho_bb_expr.doit(deep=False)}))

### Visual comparison

In [ ]:
args = (s, m1, m2)
cm_func = sp.lambdify(args, rho_cm_expr.doit())
cm_bb_func = sp.lambdify(args, rho_bb_expr.doit())
rho_func = sp.lambdify(args, rho_expr.doit())

s_values = np.linspace(0, 2, 400)
m1_val = 0.6
m2_val = 0.4
args = (m1_val, m2_val)

cm_values = cm_func(s_values, *args)
cm_bb_values = cm_bb_func(s_values, *args)
rho_values = 1j * rho_func(s_values, *args)

fig, axes = plt.subplots(figsize=(15, 6), ncols=3, sharex=True, sharey=True)
ax1, ax2, ax3 = axes
ax1.plot(s_values, cm_values.real, label="Real part")
ax1.plot(s_values, cm_values.imag, label="Imaginary part")
ax1.set_xlabel("Re(s)")
ax1.set_ylabel("Function value")
ax1.set_title("Chew Mandelstam Function")
ax1.legend()

ax2.plot(s_values, rho_values.real, label="Real part")
ax2.plot(s_values, rho_values.imag, label="Imaginary part")
ax2.set_xlabel("Re(s)")
ax2.set_title("Phase Space Factor")
ax2.legend()

ax3.plot(s_values, cm_bb_values.real, label="Real part")
ax3.plot(s_values, cm_bb_values.imag, label="Imaginary part")
ax3.set_xlabel("Re(s)")
ax3.set_title("Chew Mandelstam Function BB")
ax3.legend()

ax2.set_ylim(-1.2, +1.6)
plt.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

:::{note}
No Blatt-Weisskopf form factors because $L=0$.
:::

In [ ]:
gamma, m0, w0 = sp.symbols("gamma m0 Gamma0")
k_expr = (gamma**2 * m0 * w0) / (s - m0**2)
rho_expressions = {
    K[0, 0]: k_expr,
    rho[0, 0]: PhaseSpaceFactor(s, m1, m2),
}
cm_expressions = {
    **rho_expressions,
    rho[0, 0]: PhaseSpaceCM(s, m1, m2),
}
cm_conj_expressions = {
    **rho_expressions,
    rho[0, 0]: PhaseSpaceCM(s, m1, m2).conjugate(),
}
Math(aslatex(cm_expressions))

In [ ]:
T_rho_expr = T_explicit[0, 0].xreplace(rho_expressions)
T_cm_expr = T_explicit[0, 0].xreplace(cm_expressions)
T_cm_conj_expr = T_explicit[0, 0].xreplace(cm_conj_expressions)

In [ ]:
T_cm_expr.simplify(doit=False)

## Riemann sheet visualization

To get to the physical/unphysical Riemann sheet one uses XXXX which is derived from Schwarz reflection principle

In [ ]:
symbols = sp.Tuple(s, m1, m2, m0, w0, gamma)
T_rho_func = sp.lambdify(symbols, T_rho_expr.doit())
T_cm_func = sp.lambdify(symbols, T_cm_expr.doit())
T_cm_conj_func = sp.lambdify(symbols, T_cm_conj_expr.doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 5, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

values = {
    m1: 0.9,
    m2: 0.8,
    m0: 3.1,
    w0: 1.0,
    gamma: 1,
}
args = eval(str(symbols[1:].xreplace(values)))

Tp = T_rho_func(Zp**2, *args)
Tn = T_rho_func(Zn**2, *args)

Tp_cm = T_cm_func(Zp**2, *args)
Tn_cm = T_cm_func(Zn**2, *args)

Tp_cm_conj = T_cm_conj_func(Zp**2, *args)
Tn_cm_conj = T_cm_conj_func(Zn**2, *args)

In [ ]:
fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharey=True)
ax1, ax2 = axes
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(s)$")
ax1.set_ylabel("Function Value")

ax1.plot(x, Tp_cm[0].imag)
ax1.plot(x, Tn_cm_conj[0].imag)
ax1.set_title("Chew Mandelstam Function")

ax2.plot(x, Tp[0].imag)
ax2.plot(x, Tn[0].imag)
ax2.set_title("Phsp Function")

fig.tight_layout()
plt.show()

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    n_lines = 16
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        contours=dict(
            x=dict(
                show=True,
                start=x.min(),
                end=x.max(),
                size=(x.max() - x.min()) / n_lines,
                color="black",
            ),
            y=dict(
                show=True,
                start=-y.max(),
                end=+y.max(),
                size=(y.max() - y.min()) / (n_lines // 2),
                color="black",
            ),
        ),
        name=sheet_name,
        opacity=0.6,
        showscale=False,
    )


vmax = 1.6
project = np.imag
projection_text = "Imaginary"
sheet_colors = {
    "Unphysical": "blue",
    "Physical": "red",
}

In [ ]:
Sn = go.Surface(x=X, y=-Y, z=project(Tn), **sty("Unphysical"))
Sp = go.Surface(x=X, y=+Y, z=project(Tp), **sty("Physical"))
zeros = np.zeros(x.shape)

line_rho = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(Tp[0]),
    line={"color": "black", "width": 4},
    mode="lines",
    name="Lineshape",
)

threshold_point = go.Scatter3d(
    x=[float((m1 + m2).subs(values) ** 2)],
    y=[0],
    z=[0],
    mode="markers",
    marker={"color": "black", "size": 5},
    name="Threshold",
)
fig = go.Figure(data=[Sn, Sp, line_rho, threshold_point])
fig.update_layout(
    height=550,
    width=600,
    showlegend=False,
    title_text=(
        f"{projection_text} part of T function with standard phase space factor"
    ),
)
fig.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig.show()

In [ ]:
Sn_cm = go.Surface(x=X, y=-Y, z=project(Tn_cm), **sty("Unphysical"))
Sp_cm = go.Surface(x=X, y=+Y, z=project(Tp_cm), **sty("Unphysical"))

Sn_cm_conj = go.Surface(x=X, y=-Y, z=project(Tn_cm_conj), **sty("Physical"))
Sp_cm_conj = go.Surface(x=X, y=+Y, z=project(Tp_cm_conj), **sty("Physical"))

intersection_points = []
for i in range(len(x)):
    intersection_points.append((x[i], 0, project(Tn_cm[0])[i]))

intersection_points = np.array(intersection_points)
intersection_line = go.Scatter3d(
    x=intersection_points[:, 0],
    y=intersection_points[:, 1],
    z=intersection_points[:, 2],
    mode="lines+markers",
    line=dict(color="yellow", width=20),
    marker=dict(size=1, color="pink"),
    name="Intersection Line",
)

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "surface"}, {"type": "surface"}, {"type": "scatter3d"}]],
)

fig.add_trace(Sn_cm, row=1, col=1)
fig.add_trace(Sp_cm, row=1, col=1)

fig.add_trace(Sn_cm_conj, row=1, col=3)
fig.add_trace(Sp_cm_conj, row=1, col=3)

fig.add_trace(Sn_cm, row=1, col=2)
fig.add_trace(Sp_cm_conj, row=1, col=2)

fig.add_trace(intersection_line, row=1, col=2)

fig.update_layout(
    height=550,
    width=1200,
    showlegend=False,
    title_text=f"{projection_text} part of T function with Chew-Mandelstam",
)
fig.update_scenes(
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)
fig.show()

In [ ]:
fig, axes = plt.subplots(figsize=(15, 5), ncols=3, sharey=True)
ax1, ax2, ax3 = axes
ax1.set_title("Unphysical")
ax2.set_title("Physical")
ax3.set_title("Physical and Unphysical")
ax1.set_ylabel(R"$\mathrm{Im} s$")
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re} s$")

kwargs = dict(
    cmap=plt.cm.coolwarm,
    vmin=-vmax,
    vmax=+vmax,
)
mesh = ax1.pcolormesh(X, -Y, project(Tn_cm), **kwargs)
mesh = ax1.pcolormesh(X, +Y, project(Tp_cm), **kwargs)
mesh = ax2.pcolormesh(X, -Y, project(Tn_cm_conj), **kwargs)
mesh = ax2.pcolormesh(X, +Y, project(Tp_cm_conj), **kwargs)
mesh = ax3.pcolormesh(X, +Y, project(Tp_cm), **kwargs)
mesh = ax3.pcolormesh(X, -Y, project(Tn_cm_conj), **kwargs)
fig.colorbar(mesh, ax=ax3, label="Physical and Unphysical")

fig.tight_layout()
plt.show()

## k-plane plots 

In [ ]:
k = sp.Symbol("k")
s_to_k_expr = sp.sqrt(((s - (m1 + m2) ** 2) * (s - (m1 - m2) ** 2)) / (4 * s))
equation = sp.Eq(k, s_to_k_expr)
solutions = sp.solve(equation, s)
Math(aslatex(solutions))

In [ ]:
k_to_s_expr = solutions[1]
k_to_s_func = sp.lambdify([k, m1, m2], k_to_s_expr.doit())

In [ ]:
T_max = np.linspace(-10, +10, num=200)
ky = np.linspace(epsilon, 10, num=100)
Xk, Yk = np.meshgrid(T_max, ky)
Kn = Xk - Yk * 1j
Kp = Xk + Yk * 1j

Skn = k_to_s_func(Kn, m1_val, m2_val)
Skp = k_to_s_func(Kp, m1_val, m2_val)

Tp_cm_k = T_cm_func(Skp, *args)
Tn_cm_k = T_cm_func(Skn, *args)
Tp_cm_conj_k = T_cm_conj_func(Skp, *args)
Tn_cm_conj_k = T_cm_conj_func(Skn, *args)

fig, axes = plt.subplots(figsize=(15, 5), ncols=3, sharey=True)
ax1, ax2, ax3 = axes
ax1.set_title("Unphysical")
ax2.set_title("Physical")
ax3.set_title("Physical and Unphysical")
ax1.set_ylabel(R"$\mathrm{Im}(k)$")
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(k)$")

k_max = 1e-10
kwargs = dict(
    cmap=plt.cm.coolwarm,
    vmin=-k_max,
    vmax=+k_max,
)
mesh = ax1.pcolormesh(Xk, +Yk, project(Tn_cm_k), **kwargs)
mesh = ax1.pcolormesh(Xk, -Yk, project(Tp_cm_k), **kwargs)
mesh = ax2.pcolormesh(Xk, +Yk, project(Tn_cm_conj_k), **kwargs)
mesh = ax2.pcolormesh(Xk, -Yk, project(Tp_cm_conj_k), **kwargs)
mesh = ax3.pcolormesh(Xk, -Yk, project(Tp_cm_k), **kwargs)
mesh = ax3.pcolormesh(Xk, +Yk, project(Tn_cm_conj_k), **kwargs)
fig.colorbar(mesh, ax=ax3)

fig.tight_layout()
plt.show()